In [16]:
import numpy
from gt4py.gtscript import lazy_stencil, Field, computation, interval
from gt4py.storage import from_array

# Lazy Stencil functionality
Let's define some stencils, but defer the potentially long compile time. However we would like to be sure the compilation won't fail for some obvious reason before we try using them.

In [26]:
@lazy_stencil(backend="debug", rebuild=True, check_syntax=True)
def copy(src: Field[float], tgt: Field[float]):
    """Copy src into tgt."""
    with computation(PARALLEL), interval(...):
        tgt = src
        
        
@lazy_stencil(backend="gtmc", rebuild=True)
def copy_long_compile(src: Field[float], tgt: Field[float]):
    """Copy src into tgt."""
    with computation(PARALLEL), interval(...):
        tgt = src

Note that this executed much faster than it would have if you had used `@stencil` with the `gtmc` backend.
By the way, we can also check the syntax manually at a later point instead:

In [27]:
copy_long_compile.check_syntax()

We can still use the lazy stencil as a callable.

In [28]:
x = numpy.array([[[1.]]])
y = numpy.array([[[0.]]])
copy(x, y, origin=(0, 0, 0))
assert y[0, 0, 0] == 1.

/Users/ricoh/Code/gt4py/src/gt4py/stencil_object.py:198: RuntimeWarning: NumPy ndarray passed as field. This is discouraged and only works with constraints and only for certain backends.
  RuntimeWarning,


This will trigger the compilation, as should be noticeable in the execution time of the next cell.

In [29]:
xs = from_array(x, backend=copy_long_compile.backend.name, default_origin=(0, 0, 0))
y[0, 0, 0] = 0.
ys = from_array(y, backend=copy_long_compile.backend.name, default_origin=(0, 0, 0))
copy_long_compile(xs, ys)
assert ys[0, 0, 0] == 1.

Of course, the compilated stencil implementation is cached, so the next call will be fast again. From now calls go through directly to the compiled stencil without even a conditional check.
If you want to remove even the slightest doubt that there might be any overhead you can also directly access the compiled stencil object.

In [30]:
# cached access to underlying stencil object
ys[0, 0, 0] = 0.
copy_long_compile(xs, ys)

# direct access
copy_lc_impl = copy_long_compile.implementation
ys[0, 0, 0] = 0.
copy_lc_impl(xs, ys)

Let's see what happens if we actually have some invalid `gtscript` syntax inside the stencil definition...

In [32]:
@lazy_stencil(backend="gtmc", rebuild=True, check_syntax=True)
def copy_pasta(src: Field[float], tgt: Field[float]):
    """Copy src into tgt but with a syntax error from copy pasting."""
    with computation(PARALLEL), interval(...):
        with interval(...):
            tgt = src

GTScriptSyntaxError: Invalid 'with' statement at line 4 (column 9)